In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

Hyper Parameters 

In [2]:
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

MNIST Dataset (Images and Labels)

In [ ]:
train_dataset = dsets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = dsets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

Dataset Loader (Input Pipline)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

Model

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        out = self.linear(x)
        return out

In [ ]:
model = LogisticRegression(input_size, num_classes)


Loss and Optimizer

Softmax is <span class="burk">internally</span> computed.

Set parameters to be updated.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Training the Model

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' %
                  (epoch + 1, num_epochs, i + 1,
                   len(train_dataset) // batch_size, loss.data[0]))

Test the Model

In [ ]:
correct = 0
total = 0
for images, labels in test_loader:
    images = images.view(-1, 28 * 28)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the model on the 10000 test images: %d %%' %
      (100 * correct / total))

Save the Model

In [ ]:
torch.save(model.state_dict(), 'model.pkl')